In [2]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format ='retina'
%matplotlib inline
import sys,os
sys.path.append('../../')

import numpy as np
import matplotlib.pyplot as plt
import corner
from ceffyl import Ceffyl, models, Sampler, chain_utils
from enterprise.signals import parameter
# import ultranest

import la_forge.core as co
import la_forge.diagnostics as dg

import natpy as nat

# from jax import config
# config.update("jax_enable_x64", True)
# import jax
# import jax.numpy as jnp


Optional mpi4py package is not installed.  MPI support is not available.


In [7]:
data_dir = './NG15_Ceffyl/30f_fs{hd}_ceffyl/'
pta = Ceffyl.ceffyl(datadir=data_dir)
gw = Ceffyl.signal(psd=models.powerlaw, N_freqs=5)
pta.add_signals([gw])
print(pta.Tspan)

505861299.1401643


In [ ]:
params_dict = {'log10_A': -14, 'gamma': 3}
# pta.ln_likelihood(params )

print(pta.signals[0].param_names)

for s in pta.signals:
    print(s.param_names)

x0 = pta.initial_samples()
print(x0)
print(pta.ln_likelihood(x0))
params = np.array([-14.,3.])


params = np.concatenate([[params], [x0]])
print(params.shape)
print(params)
# print(pta.ln_likelihood(params))


['gw_log10_A', 'gw_gamma']
['gw_log10_A', 'gw_gamma']
[-16.83483622   0.77957284]
-84.87087067196366
(2, 2)
[[-14.           3.        ]
 [-16.83483622   0.77957284]]


ValueError: non-broadcastable output operand with shape (1,5) doesn't match the broadcast shape (2,5)

Conversion between OmegaGW and rho (sampled by Ceffyl)

$h^2 \Omega_{\rm GW} = (8 \pi^4 f^5 h^2 ) \times 10^{(2\log_{10}\rho +\log_{10}T_{\rm span}) } / H_0^2$

$T_{\rm span}\rho^2 = \dfrac{\Omega_{\rm GW}H_0^2}{8 \pi^4 f^5}$

In [14]:
# convert to OmegaGW
Tspan = 15 * 365 * 24 * 3600
h = 0.672
# print(data_list.shape)
H_0  = h * 100 * nat.convert(nat.km * nat.s**-1 * nat.Mpc**-1, nat.GeV) # Hubble constant (GeV)
H_0_Hz  = H_0 * nat.convert(nat.GeV, nat.Hz)
# print(H_0_Hz)
# OmegaGW_data = (8*np.pi**4 * freqs[:,None]**5 * h**2 * 10**(2 * data_list + np.log10(Tspan)) ) / (H_0_Hz**2)
# OmegaGW_data  = np.log10(OmegaGW_data)

from scipy.interpolate import CubicSpline

left_node = -9.
right_node = -7.

def interpolate(nodes,vals,x):
    # nodes = jnp.pad(nodes, (1,1), 'constant', constant_values=(left_node, right_node))
    spl = CubicSpline(nodes,vals,check=False)
    res = np.power(10,spl(x))
    res = np.where(x<left_node, 0, res)
    res = np.where(x>right_node, 0, res)
    return res

frequencies = np.load(f'{data_dir}/freqs.npy')
print(frequencies)

from gwb.omega_gw_jax import OmegaGWJax
s = np.linspace(0, 1, 15)  # First rescaled internal momentum
t = np.logspace(-5,5, 250)  # Second rescaled internal momentum
t_expanded = np.expand_dims(t, axis=-1)
## Repeat t along the new axis to match the shape (100, 1000)
t = np.repeat(t_expanded, len(frequencies), axis=-1)

gwb_calculator = OmegaGWjax(s=s,t=t,f=frequencies,norm="RD",jit=True)

def get_gwb(nodes,vals):
    pf = lambda k: interpolate(nodes,vals,np.log10(k))
    omegagw = gwb_calculator(pf,frequencies)
    return (omegagw,)

get_gwb_func = jax.jit(get_gwb)



[1.97682646e-09 3.95365292e-09 5.93047937e-09 7.90730583e-09
 9.88413229e-09 1.18609587e-08 1.38377852e-08 1.58146117e-08
 1.77914381e-08 1.97682646e-08 2.17450910e-08 2.37219175e-08
 2.56987439e-08 2.76755704e-08 2.96523969e-08 3.16292233e-08
 3.36060498e-08 3.55828762e-08 3.75597027e-08 3.95365292e-08
 4.15133556e-08 4.34901821e-08 4.54670085e-08 4.74438350e-08
 4.94206614e-08 5.13974879e-08 5.33743144e-08 5.53511408e-08
 5.73279673e-08 5.93047937e-08]


AttributeError: partially initialized module 'jax' has no attribute 'version' (most likely due to a circular import)